<a href="https://colab.research.google.com/github/Yi-Wei-Lin/Tibame_AI_Project/blob/main/Cabbage_predict_0707.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [122]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [123]:
data = pd.read_excel("Cabbage.xlsx")

In [124]:
data = data.dropna()
data = data.loc[data["Market"] == "台北一"]
data

,Date,Market,Product,Up_price,Mid_price,Low_price,Avg_price,Volume,Month,Week_day,Year,Rest_day
39204,2000-01-02,台北一,LA1 甘藍 初秋,11.0,8.0,5.0,8.9,192029.0,1,7,2000,0
39205,2000-01-03,台北一,LA1 甘藍 初秋,9.0,7.0,4.0,7.2,174270.0,1,1,2000,0
39206,2000-01-04,台北一,LA1 甘藍 初秋,8.0,6.0,3.0,6.7,125426.0,1,2,2000,0
39207,2000-01-05,台北一,LA1 甘藍 初秋,10.0,7.0,4.0,7.1,119318.0,1,3,2000,0
39208,2000-01-06,台北一,LA1 甘藍 初秋,9.0,7.0,5.0,6.8,118178.0,1,4,2000,0
...,...,...,...,...,...,...,...,...,...,...,...,...
47037,2021-06-13,台北一,LA1 甘藍 初秋,23.0,15.6,6.2,15.2,174973.0,6,7,2021,0
47038,2021-06-14,台北一,LA1 甘藍 初秋,27.4,21.4,13.6,21.0,128693.0,6,1,2021,0
47041,2021-06-17,台北一,LA1 甘藍 初秋,21.8,15.0,8.5,15.1,96289.0,6,4,2021,0
47042,2021-06-18,台北一,LA1 甘藍 初秋,21.9,15.2,8.1,15.1,148701.0,6,5,2021,0


In [125]:
data_train = data[data["Date"] < "2019-01-01"]
data_test = data[data["Date"] >= "2019-01-01"]

In [126]:
data_train = data_train.drop(["Date", "Market", "Product", "Month", "Week_day", "Year", "Rest_day"], axis=1)
data_train

,Up_price,Mid_price,Low_price,Avg_price,Volume
39204,11.0,8.0,5.0,8.9,192029.0
39205,9.0,7.0,4.0,7.2,174270.0
39206,8.0,6.0,3.0,6.7,125426.0
39207,10.0,7.0,4.0,7.1,119318.0
39208,9.0,7.0,5.0,6.8,118178.0
...,...,...,...,...,...
46137,10.5,5.9,2.5,6.1,170578.0
46138,9.4,5.8,2.7,5.9,170145.0
46139,9.6,5.2,2.8,5.6,176992.0
46140,10.2,5.5,2.4,5.8,195142.0


In [127]:
# from tensorflow.keras.utils import to_categorical
# pd.DataFrame(to_categorical(data_train["Month"]))

In [128]:
scaler = MinMaxScaler()
data_train = scaler.fit_transform(data_train)

In [129]:
x_train = []
y_train = []
time_step = 60
for i in range(len(data_train) - time_step):
  x_train.append(data_train[i:(i+time_step)])
  y_train.append(data_train[(i+time_step)][3])
x_train, y_train = np.array(x_train), np.array(y_train)

In [130]:
# Building LSTM model
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
layers = [
    LSTM(units=60, activation="relu", return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])),
    Dropout(0.2),
    LSTM(units=80, activation="relu", return_sequences=True),
    Dropout(0.2),
    LSTM(units=100, activation="relu", return_sequences=True),
    Dropout(0.2),
    LSTM(units=120, activation="relu"),
    Dropout(0.2),
    Dense(units=1)
]
regressor = Sequential(layers)
regressor.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_8 (LSTM)                (None, 60, 60)            15840     
_________________________________________________________________
dropout_8 (Dropout)          (None, 60, 60)            0         
_________________________________________________________________
lstm_9 (LSTM)                (None, 60, 80)            45120     
_________________________________________________________________
dropout_9 (Dropout)          (None, 60, 80)            0         
_________________________________________________________________
lstm_10 (LSTM)               (None, 60, 100)           72400     
_________________________________________________________________
dropout_10 (Dropout)         (None, 60, 100)           0         
_________________________________________________________________
lstm_11 (LSTM)               (None, 120)              

In [131]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
regressor.compile(loss = 'mean_squared_error',
         optimizer = Adam()                   
)
regressor.fit(x_train, y_train, epochs=2, batch_size=32)

Epoch 1/2
179/179 [==============================] - 97s 515ms/step - loss: 0.0101
Epoch 2/2
179/179 [==============================] - 92s 515ms/step - loss: 0.0052


In [159]:
past_60_days = data[data["Date"] < "2019-01-01"].tail(time_step)
df = past_60_days.append(data_test, ignore_index = True)
df = df.drop(["Date", "Market", "Product", "Month", "Week_day", "Year", "Rest_day"], axis=1)
df

KeyError: ignored

In [150]:
inputs = scaler.transform(df)
inputs[]

array([[0.42256637, 0.35260931, 0.34716981, 0.37080292, 0.14097223],
       [0.39712389, 0.32157969, 0.27169811, 0.33284672, 0.13059995],
       [0.38163717, 0.33286319, 0.2490566 , 0.33284672, 0.11554971],
       ...,
       [0.24115044, 0.1819464 , 0.16037736, 0.18832117, 0.25345699],
       [0.24225664, 0.18476728, 0.15283019, 0.18832117, 0.39182097],
       [0.23340708, 0.19040903, 0.17735849, 0.19416058, 0.40467743]])

In [151]:
x_test = []
y_test = []
time_step = 60
for i in range(len(inputs) - time_step):
  x_test.append(inputs[i:(i+time_step)])
  y_test.append(inputs[(i+time_step)][3])
yy = y_test
x_test, y_test = np.array(x_test), np.array(y_test)

In [152]:
x_test.shape, y_test.shape
scaler.scale_[3]
scale = 1 / scaler.scale_[3]

In [140]:
scaler.scale_
# 1 / scaler.scale_[3]

array([1.10619469e-02, 1.41043724e-02, 1.88679245e-02, 1.45985401e-02,
       2.63992946e-06])

In [166]:
pre_price = regressor.predict(x_test)
# pre_price = pre_price.reshape(731,60)
# pre_price = np.reshape(pre_price, (-1,1))
# pre_price = pre_price * scale
# y_test = y_test * scale
pre_price*scale
xx = y_test*scale
pre_price.shape
# nsamples, nx, ny = x_test.shape
# d2_x_test = x_test.reshape((nsamples,nx*ny))
# d2_x_test.shape

(731, 1)

In [176]:
# y_pred_reshaped = np.zeros((len(pre_price), 5))
# y_pred_reshaped[:,3] = pre_price[0]
# y_pred = scaler.inverse_transform(y_pred_reshaped)
# # y_pred_reshaped
# y_pred

ValueError: ignored

In [137]:
# x_test.shape
# scaler.inverse_transform(x_test)
# pre_price

ValueError: ignored

In [ ]:
# Visualising the results
plt.figure(figsize=(14,5))
plt.plot(y_test, color = 'red', label = 'Real Cabbage Price')
plt.plot(pre_price, color = 'blue', label = 'Predicted Cabbage Price')
plt.title('Cabbage Price Prediction')
plt.xlabel('Time')
plt.ylabel('Cabbage Price')
plt.legend()
plt.show()

In [ ]:
len(y_test)
y_test[730]